In [ ]:
import torch
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
model = torch.load("Conformer_tiny_patch16.pth", map_location='cpu')


In [ ]:
model_keys = []
for key in model.keys():
    if "num_batches_tracked" not in key:
        model_keys.append(key)
        print(key)

In [ ]:
weights = []
prefix = "model."
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix +  key.replace("qkv", "q")
        key_k = prefix +  key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    else:
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})
        print(key, weight.shape)

In [ ]:
weights_out = []
for weight in weights:
    weight_out = {}
    name = weight["name"]
    if "conv_trans_" in weight["name"]:
        name = weight["name"].replace("conv_trans_", "conv_trans.").split(".")
        name[2] = str(int(name[2])-2)
        name = ".".join(name)
    weight_out["name"] = name
    weight_out["data"] = weight["data"]
    weights_out.append(weight_out)
    print(weight_out["name"])


In [ ]:
save_checkpoint(weights_out, "Conformer_tiny_patch16.ckpt")

In [ ]:
len(weights_out)